In [1]:
# import necessary packages
import sys, os, glob
import torch
import matplotlib.pyplot as plt

from torch_geometric.nn import Node2Vec
from sklearn.manifold import TSNE
from torch_geometric.datasets import Planetoid
from tqdm.notebook import tqdm
from importlib import reload

sys.path.append('../')

# custom imports
from utils.GetLowestGPU import GetLowestGPU
from utils.GetFileNames import get_file_names
import utils.preprocessing as pp
device = GetLowestGPU()

## Load Data

In [9]:
# options
network_path = "../data/networks/DREAM4/DREAM4_in-silico_challenge/Size_10/DREAM4_gold_standards/"
graph_names = get_file_names(network_path, ".tsv")

In [10]:
# view graph names
graph_names

['insilico_size10_4_goldstandard.tsv',
 'insilico_size10_5_goldstandard.tsv',
 'insilico_size10_1_goldstandard.tsv',
 'insilico_size10_3_goldstandard.tsv',
 'insilico_size10_2_goldstandard.tsv']

In [14]:
# convert graphs to node/edge lists and then nx objects
graphs = []
for name in tqdm(graph_names):
    # convert to nx object and append
    graphs.append(pp.convert_to_nx(network_path + name))
print(f"Processed {len(graphs)} graphs!")



  0%|          | 0/5 [00:00<?, ?it/s]

Processed 5 graphs!


Pytorch Geometric has a nice [tutorial](https://colab.research.google.com/github/AntonioLonga/PytorchGeometricTutorial/blob/main/Tutorial11/Tutorial11.ipynb)

## Node2Vec

* pytorch geometric has [node2vec](https://pytorch-geometric.readthedocs.io/en/latest/tutorial/shallow_node_embeddings.html)

In [18]:
# instantiate node2vec model on dummy data
dataset = "Cora"
path = os.path.join("../data/test/", dataset)
dataset = Planetoid(path, dataset)
data = dataset[0]

Processing...
Done!
/mnt/DGX01/Personal/krusepi/.venv/lib/python3.9/site-packages/torch_geometric/io/fs.py:215: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.lo

tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])

In [ ]:
model = Node2Vec(data.edge_index, embedding_dim=128, 
                 walk_length=20,                        # lenght of rw
                 context_size=10, walks_per_node=20,
                 num_negative_samples=1, 
                 p=200, q=1,                             # bias parameters
                 sparse=True).to(device)

### Create Dataloader

In [ ]:
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
for idx, (pos_rw, neg_rw) in enumerate(loader):
    print(idx, pos_rw.shape, neg_rw.shape)

## Graph AutoEncoder

* torch geometric also has [GAE](https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.models.GAE.html)

## GCN

## GraphLLM Embedding

* As seen in [GraphLLM Paper](https://arxiv.org/abs/2310.05845)

## Multiverse (Maybe)